In [19]:
import pandas as pd

In [30]:
#generate all possible time frames from a list of years
def generate_time_frames(years):
    time_frames = []
    for i in range(len(years)-1):
        for j in range(i+1, len(years)):
            time_frames.append(f"{years[i]}-{years[j]}")
    return time_frames
def calculate_changes(df, columns, time_frames, years):
    for column in columns:
        for time_frame in time_frames:
            start_year, end_year = time_frame.split('-')
            df[f'{column} % Change', 'None', f'{time_frame}'] = percentchange(df[(column, int(end_year), 'None')], df[(column, int(start_year), 'None')])
            df[f'{column} Change', 'None', f'{time_frame}'] = (df[(column, int(end_year), 'None')] - df[(column, int(start_year), 'None')])

    return df
#functions
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y

In [34]:
data = pd.read_csv('Phones.csv')
data.head(3)

,Year,Model,Tenure,Enrolled Devices,Claims,Device IR
0,2019,iPhone XS,4,250.0,29.0,0.116000
1,2019,Galaxy S9,10,292.0,15.0,0.051429
2,2019,iPhone XS Max,4,175.0,24.0,0.137143


In [35]:
data['Device IR'] = round(data['Device IR'] * 100, 1)

In [36]:
data.head()

,Year,Model,Tenure,Enrolled Devices,Claims,Device IR
0,2019,iPhone XS,4,250.0,29.0,11.6
1,2019,Galaxy S9,10,292.0,15.0,5.1
2,2019,iPhone XS Max,4,175.0,24.0,13.7
3,2019,iPhone X,14,358.0,13.0,3.6
4,2019,Google Pixel 3,3,158.0,26.0,16.4


In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              17 non-null     int64  
 1   Model             17 non-null     object 
 2   Tenure            17 non-null     int64  
 3   Enrolled Devices  16 non-null     float64
 4   Claims            16 non-null     float64
 5   Device IR         16 non-null     float64
dtypes: float64(3), int64(2), object(1)
memory usage: 944.0+ bytes


In [38]:
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [39]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('Model')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'Model', columns = ['Year'], values = cols)
df_pivot.head(2)

Claims             Device IR             Enrolled Devices  \
Year             2019  2020  2021      2019  2020  2021             2019   
Model                                                                      
Galaxy S9        15.0  23.0  22.0       5.1   7.2   7.2            292.0   
Google Pixel 3   26.0  26.0  31.0      16.4  14.9  11.6            158.0   

                             Tenure              
Year             2020   2021   2019  2020  2021  
Model                                            
Galaxy S9       321.0  305.0   10.0  22.0  34.0  
Google Pixel 3  174.0  268.0    3.0  15.0  27.0

In [40]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

Claims             Device IR             Enrolled Devices  \
                    2019  2020  2021      2019  2020  2021             2019   
                    None  None  None      None  None  None             None   
Model                                                                         
Galaxy S9           15.0  23.0  22.0       5.1   7.2   7.2            292.0   
Google Pixel 3      26.0  26.0  31.0      16.4  14.9  11.6            158.0   
iPhone 11 Pro Max    NaN  13.0   NaN       NaN  14.2   NaN              NaN   

                                Tenure              
                    2020   2021   2019  2020  2021  
                    None   None   None  None  None  
Model                                               
Galaxy S9          321.0  305.0   10.0  22.0  34.0  
Google Pixel 3     174.0  268.0    3.0  15.0  27.0  
iPhone 11 Pro Max   92.0    NaN    NaN   4.0  16.0

In [41]:
#get a list of the variables to loop through by indexing into the first level only of the column headers
first_level = df_pivot.columns.get_level_values(0).unique().tolist()

In [42]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [43]:
#reformat and rename columns
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [51]:
data

,Model,Year,Time Frame,Claims,Claims % Change,Claims Change,Device IR,Device IR % Change,Device IR Change,Enrolled Devices,Enrolled Devices % Change,Enrolled Devices Change,Tenure,Tenure % Change,Tenure Change
0,Galaxy S9,2019,None,15.0,NaN,NaN,5.1,NaN,NaN,292.0,NaN,NaN,10.0,NaN,NaN
1,Galaxy S9,2020,None,23.0,NaN,NaN,7.2,NaN,NaN,321.0,NaN,NaN,22.0,NaN,NaN
2,Galaxy S9,2021,None,22.0,NaN,NaN,7.2,NaN,NaN,305.0,NaN,NaN,34.0,NaN,NaN
3,Galaxy S9,None,2019-2020,NaN,53.333333,8.0,NaN,41.176471,2.1,NaN,9.931507,29.0,NaN,120.000000,12.0
4,Galaxy S9,None,2019-2021,NaN,46.666667,7.0,NaN,41.176471,2.1,NaN,4.452055,13.0,NaN,240.000000,24.0
5,Galaxy S9,None,2020-2021,NaN,-4.347826,-1.0,NaN,0.000000,0.0,NaN,-4.984424,-16.0,NaN,54.545455,12.0
6,Google Pixel 3,2019,None,26.0,NaN,NaN,16.4,NaN,NaN,158.0,NaN,NaN,3.0,NaN,NaN
7,Google Pixel 3,2020,None,26.0,NaN,NaN,14.9,NaN,NaN,174.0,NaN,NaN,15.0,NaN,NaN
8,Google Pixel 3,2021,None,31.0,NaN,NaN,11.6,NaN,NaN,268.0,NaN,NaN,27.0,NaN,NaN
9,Google Pixel 3,None,2019-2020,NaN,0.000000,0.0,NaN,-9.146341,-1.5,NaN,10.126582,16.0,NaN,400.000000,12.0


In [47]:
first = data.loc[(data['Model'] == 'iPhone XS Max') & (data['Time Frame'] != 'None')]
first = first[['Model', 'Time Frame', 'Enrolled Devices % Change', 'Enrolled Devices Change']]
first

,Model,Time Frame,Enrolled Devices,Enrolled Devices % Change,Enrolled Devices Change
30,iPhone XS Max,2019-2020,NaN,18.857143,33.0
31,iPhone XS Max,2019-2021,NaN,10.285714,18.0
32,iPhone XS Max,2020-2021,NaN,-7.211538,-15.0


In [50]:
iph11pm = data.loc[data['Model'] == 'iPhone 11 Pro Max'].reset_index(drop = True)
iph11pm

,Model,Year,Time Frame,Claims,Claims % Change,Claims Change,Device IR,Device IR % Change,Device IR Change,Enrolled Devices,Enrolled Devices % Change,Enrolled Devices Change,Tenure,Tenure % Change,Tenure Change
0,iPhone 11 Pro Max,2020,None,13.0,NaN,NaN,14.2,NaN,NaN,92.0,NaN,NaN,4.0,NaN,NaN
1,iPhone 11 Pro Max,2021,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN
2,iPhone 11 Pro Max,None,2020-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,12.0


The iPhone 11 Pro Max didn't exist or wasn't tracked in 2019 so we will impute this data based on the rate of change of the iPhone XS Max between 2020 and 2021. I'm also choosing to use the rate of change instead of the real change because it's a more relative measurement.

In [ ]:
print("The iPhone 11 Pro Max enrolled subscribers changed by -15% between 2020 and 2021 resulting in" + str())